# WebComponent Reference

In [ ]:
import param
import panel as pn
pn.extension()
pn.config.sizing_mode="stretch_width"
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

WebComponent
============

Use the WebComponent to quickly plugin webcomponents and/ or Javascript libraries not already
supported in Panel.

You can use it by instantiating an instance or inheriting from it.

Parameters
---------------

- The parameter values listed in `attributes_to_watch` will be used to set the `html` parameter
attributes on construction. Further more the the key attributes of the Javascript
WebComponent and the value parameters of the Python WebComponent will be kept in sync.
- The value parameters listed in `properties_to_watch` will be used to set the Javascript Web
Component key properties on construction and kept in sync after construction.
- The parameters listed in `parameters_to_watch` will be used to set the `html` parameter on
construction and when changed.
- The event keys listed in `events_to_watch` will be watched on the JavaScript side. When fired
the javascript code will check whether any properties_to_watch have changed and if yes then they
will be synced to Python. If a Python value parameter is specified it will be
incremented by +1 for each event.
- The `ColumnDataSource` is used to efficiently transfer columnar data to the javascript side.
    - The `column_data_source_orient` is used to specify how the js web component would like to receive the data.
    For now `dict` and `records` are supported.
    - The `column_data_source_load_function` specifies the name of the function on the web component that will load the data

Other
-------

- If the JavaScript WebComponent contains an `after_layout` function this can be used to
resize the JS WebComponent. An example is the ECharts web component.
- You can specify nested JS properties as values in properties_to_watch
- Please note that you will propably have to experiment a bit in order to determine which
javascript files to import and what combination of attributes, properties, events and/ or
parameters to watch.
___

## Introduction

Web Components powers the modern web. For an introduction and overview see [Web Components: The Secret Ingredient Helping Power The Web](https://www.youtube.com/watch?v=YBwgkr_Sbx0).

You can find web components to use in your Panel Project at 

- [webcomponent.org](https://www.webcomponents.org/)
- [npm](https://www.npmjs.com/), 

Please note picking and using the right web component library and the right version can be **challenging** as the web component standard is still evolving. You might have to do some experimentation before it works for you.


## Google Model Viewer

Google has developed a `model-viewer` web component for interactively viewing very large and detailed 3D models. For an introduction see [modelviewer.dev](https://modelviewer.dev/)

We start out by defining the *javascript* nescessary to load the package and the web component *html* string.

##### TODO: MAYBE REMOVE SOME OF THE JS TO SIMPLIFY?

In [ ]:
js = """
<script src="https://unpkg.com/@webcomponents/webcomponentsjs@2.2.7/webcomponents-loader.js"></script>
<script type="module" src="https://unpkg.com/@google/model-viewer/dist/model-viewer.js"></script>
<script nomodule src="https://unpkg.com/@google/model-viewer/dist/model-viewer-legacy.js"></script>
"""
js_pane = pn.pane.HTML(js)

html="""
<model-viewer src="https://modelviewer.dev/shared-assets/models/Astronaut.glb" alt="A 3D model of an astronaut"
auto-rotate camera-controls style="height:400px;width:100%;">
</model-viewer>
"""

### Simple Version

In [ ]:
model_viewer_pane = pn.pane.WebComponent(html=html, height=400, width=300)

pn.Column(
    js_pane,
    model_viewer_pane,
    height=400,
)

### Advanced Version

In [ ]:
astronaut_src = "https://modelviewer.dev/shared-assets/models/Astronaut.glb"
flight_helmet_src="https://modelviewer.dev/shared-assets/models/glTF-Sample-Models/2.0/FlightHelmet/glTF/FlightHelmet.gltf"

class ModelViewer(pn.pane.WebComponent):
    """A Wired ModelViewer"""
    html = param.String(html)
    attributes_to_watch= param.Dict({"src": "src"})
    properties_to_watch= param.Dict({"exposure": "exposure"})

    src = param.String(astronaut_src)
    exposure = param.Number(1.0, bounds=(0, 2))


model_viewer = ModelViewer(min_height=200, width=300, src=flight_helmet_src)

pn.Row(
    js_pane,
    model_viewer, pn.Param(model_viewer, parameters=["exposure"]),
)

For an even more interactive version that has been developed by Google see [model-viewer tester](https://modelviewer.dev/examples/tester.html).

## ECharts/ PyeChart

[ECharts](https://www.echartsjs.com/en/index.html) is an open-sourced JavaScript
visualization tool, which can run fluently on PC and mobile devices.
It is compatible with most modern Web Browsers.

ECharts depends on ZRender, a graphic rendering engine, to
create intuitive, interactive, and highly-customizable charts.

[Pyecharts](https://pyecharts.org/#/en-us/) is a Python api for using ECharts in Python
including Standalone, Flask, Django and Jupyter Notebooks.

In [ ]:
ECHARTS_JS = """
<script src="https://cdn.bootcss.com/echarts/3.7.2/echarts.min.js"></script>
"""

echarts_js_pane = pn.pane.HTML(ECHARTS_JS)

ECHARTS_HTML = """
    <div class="echart" style="width:100%;height:100%;"></div>
    <script type="text/javascript">
        var myScript = document.currentScript;
        var myDiv = myScript.parentElement.firstElementChild;
        var myChart = echarts.init(myDiv);
        myDiv.eChart = myChart;
        Object.defineProperty(myDiv, 'option', {
           get: function() { return null; },
           set: function(val) { this.eChart.setOption(val); this.eChart.resize();}
        });
        myDiv.after_layout = myChart.resize; // Resizes the chart after layout of parent element
    </script>"""

class EChart(pn.pane.WebComponent):
    html = param.String(ECHARTS_HTML)
    properties_to_watch = param.Dict({"option": "option"})

    echart = param.Parameter()
    option = param.Dict()

    def __init__(self, **params):
        if "echart" in params:
            params["option"] = self._to_echart_dict(params["echart"])
        super().__init__(**params)

    @classmethod
    def _to_echart_dict(cls, echart):
        if isinstance(echart, dict):
            return echart
        if 'pyecharts' in sys.modules:
            import pyecharts
            if isinstance(echart, pyecharts.charts.chart.Chart):
                return json.loads(echart.dump_options())

        return {}

    @param.depends("echart", watch=True)
    def update(self):
        self.option = self._to_echart_dict(self.echart)

In [ ]:
pn.config.js_files["echart"]="https://cdn.bootcss.com/echarts/3.7.2/echarts.min.js"
ECHART_DICT = {
    "title": {"text": "ECharts entry example"},
    "tooltip": {},
    "legend": {"data": ["Sales"]},
    "xAxis": {"data": ["shirt", "cardign", "chiffon shirt", "pants", "heels", "socks"]},
    "yAxis": {},
    "series": [{"name": "Sales", "type": "bar", "data": [5, 20, 36, 10, 10, 20]}],
}
pn.Column(
    EChart(echart=ECHART_DICT, height=500),
)

# @Philippfr. Does not work now. But works if you `.show()`

## Open Source Web Component Libraries

- [Material](https://github.com/material-components/material-components-web-components#readme) (Work in Progress)
- [Vaadin](https://vaadin.com/components) (Mature)

## Including Prebuilt Web Component Modules using unpkg.com

If you find a web component library on [npm](https://npmjs.com) you can find a corresponding precombiled library on [unpkg](https://unpkg.com)

For example if you find the `wired-button` at [https://www.npmjs.com/package/wired-button](https://www.npmjs.com/package/wired-button) then you can browse the precompiled files at [https://www.unpkg.com/browse/wired-button/](https://www.unpkg.com/browse/wired-button/) to locate the relevant precombiled file at [https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js](https://www.unpkg.com/wired-button@2.0.0/lib/wired-button.js).

## References

- [Build an app with WebComponents in 9 minutes](https://www.youtube.com/watch?v=mTNdTcwK3MM)
- [How to use Web Components in a JavaScript project](https://www.youtube.com/watch?v=88Sa-SlHRxk&t=63s)